<a href="https://colab.research.google.com/github/olcaykursun/ML/blob/main/neuralnets/transfer_learning_answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load the MobileNetV2 using only a subset of the network to reduce training time
base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3), alpha=0.35)

# Select only part of the model
base = models.Model(inputs=base.input, outputs=base.get_layer('block_3_expand_relu').output, name='my_base_model')

base.trainable = False

# Build the new model
model = models.Sequential(name='my_target_model')

# Add augmentation layers
model.add(layers.InputLayer(input_shape=(32, 32, 3)))  #Need this because of the augmentation layer(s) that can help with small data
model.add(layers.RandomRotation(0.1)) #Horizontal or vertical flip is not applicable for digits but other augmentation layers can be

# Add base layers
model.add(base)

# Add more layers as needed
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPool2D())
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Add a new axis (color channel with 1 dimension) as the first step: Last dimension of a grayscale image should be size 1.
x_train = tf.expand_dims(x_train, axis=-1)
x_test = tf.expand_dims(x_test, axis=-1)

# Resize to 32x32
x_train = tf.image.resize(x_train, [32, 32])
x_test = tf.image.resize(x_test, [32, 32])

# Convert to color - the last axis will be expanded to have 3 colors channels
x_train = tf.image.grayscale_to_rgb(x_train)
x_test = tf.image.grayscale_to_rgb(x_test)

# Process the channels as the transferred network expects, some sort of normalization (subtract the mean and divide by standard dev etc.)
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

# Adjust the class labels for multi-class setting
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Train the model
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc:.4f}')

Model: "my_target_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_rotation (RandomRot  (None, 32, 32, 3)         0         
 ation)                                                          
                                                                 
 my_base_model (Functional)  (None, 8, 8, 48)          4672      
                                                                 
 conv2d (Conv2D)             (None, 6, 6, 64)          27712     
                                                                 
 max_pooling2d (MaxPooling2  (None, 3, 3, 64)          0         
 D)                                                              
                                                                 
 global_average_pooling2d (  (None, 64)                0         
 GlobalAveragePooling2D)                                         
                                                   

In [ ]:
base.summary()

Model: "my_base_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 16, 16, 16)           432       ['input_2[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 16, 16, 16)           64        ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 16, 16, 16)           0         ['bn_Conv1[0][0]']

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load the VGG16 model, using only a subset of the network to reduce training time
base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Select only part of the base model
base = models.Model(inputs=base.input, outputs=base.get_layer('block2_pool').output, name='my_vgg_base_block2_pool')
base.summary()

# Freeze the base layers for preserving them while retraining on the new dataset
base.trainable = False

# Partially unfreeze some layers for fine-tuning if need be
#for layer in base.layers[-3:]:
#    layer.trainable = True


# Build the new model
model = models.Sequential(name='target_model_for_mnist')

# Add the VGG16 base
model.add(base)

# Add more layers as needed
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.2))  # Regularization to prevent overfitting
model.add(layers.Dense(10, activation='softmax'))

# Compile the model and adjust the learning rate if needed (it is typically auto-adjusted)
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Add a new axis (color channel with 1 dimension) as the first step: Last dimension of a grayscale image should be size 1.
x_train = tf.expand_dims(x_train, axis=-1)
x_test = tf.expand_dims(x_test, axis=-1)

# Resize to 32x32
x_train = tf.image.resize(x_train, [32, 32])
x_test = tf.image.resize(x_test, [32, 32])

# Convert to color - the last axis will be expanded to have 3 colors channels
x_train = tf.image.grayscale_to_rgb(x_train)
x_test = tf.image.grayscale_to_rgb(x_test)

# Process the channels as the transferred network expects, some sort of normalization (subtract the mean and divide by standard dev etc.)
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

# Adjust the class labels for multi-class setting
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Train the model
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc:.4f}')

2024-11-18 10:10:27.004745: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-11-18 10:10:27.004770: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-11-18 10:10:27.004775: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-11-18 10:10:27.004805: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-18 10:10:27.004817: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "my_vgg_base_block2_pool"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 1

2024-11-18 10:10:28.098759: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


938/938 [==============================] - 12s 13ms/step - loss: 1814.6436 - accuracy: 0.8717 - val_loss: 183.2947 - val_accuracy: 0.9805
Epoch 2/5
938/938 [==============================] - 12s 12ms/step - loss: 451.5113 - accuracy: 0.9597 - val_loss: 115.0823 - val_accuracy: 0.9877
Epoch 3/5
938/938 [==============================] - 12s 12ms/step - loss: 349.3305 - accuracy: 0.9685 - val_loss: 139.4011 - val_accuracy: 0.9844
Epoch 4/5
938/938 [==============================] - 12s 12ms/step - loss: 311.9524 - accuracy: 0.9712 - val_loss: 126.9877 - val_accuracy: 0.9866
Epoch 5/5
313/313 [==============================] - 3s 9ms/step - loss: 123.2404 - accuracy: 0.9843
Test accuracy: 0.9843
